In [ ]:
import math
import os
import matplotlib
import imghdr
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
from keras.applications.xception import Xception, preprocess_input
from keras.optimizers import Adam
#from keras.preprocessing import image
import keras.utils as image
from keras.losses import categorical_crossentropy
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [ ]:
dataset_root = 'C:\\DatasetD\\ROOP_100'
result_root = "C:\\DatasetD\\MyModel"
classes = ["real", "fake"]
num_classes = 2

epochs_pre = 10
epochs_fine = 20
batch_size_pre = 4
batch_size_fine = 4
lr_pre = 1e-3
lr_fine = 1e-4
snapshot_period_pre = 5
snapshot_period_fine = 5
split = 0.7